In [51]:
import pandas as pd
from download import acs5

In [6]:
acs5 = acs5.rename(columns={'index': 'censusgeo'})

In [12]:
acs5["censusgeo"] = acs5["censusgeo"].astype(str)
acs5["censusgeo"] = acs5["censusgeo"].str.split(',')

In [24]:
acs5['county_name'] = acs5['censusgeo'].apply(lambda x: x[2])
acs5['county_name'] = acs5['county_name'].apply(lambda x: x.strip())

acs5['bg_GEOID'] = acs5['GEO_ID'].apply(lambda x: x[9:])

In [46]:
print(acs5.shape)
acs5.head()

(9691, 4)


,censusgeo,GEO_ID,county_name,bg_GEOID
0,"[['Block Group 3', ' Census Tract 101', ' Je...",1500000US170830101003,' Jersey County',170830101003
1,"[['Block Group 4', ' Census Tract 101', ' Je...",1500000US170830101004,' Jersey County',170830101004
2,"[['Block Group 1', ' Census Tract 105', ' Je...",1500000US170830105001,' Jersey County',170830105001
3,"[['Block Group 2', ' Census Tract 102', ' Je...",1500000US170830102002,' Jersey County',170830102002
4,"[['Block Group 3', ' Census Tract 103', ' Je...",1500000US170830103003,' Jersey County',170830103003


In [3]:
# read shapefiles 
blockgroups = gpd.read_file('bg_shape/tl_2018_17_bg.shp')
places = gpd.read_file('places_shape/tl_2018_17_place.shp')

blockgroups = blockgroups[['GEOID', 'NAMELSAD', 'geometry']]
places = places[['GEOID', 'NAME', 'NAMELSAD', 'geometry']]

# merge dataset
blockgroups_places = gpd.sjoin(blockgroups, places, how="inner", op="intersects")

In [4]:
blockgroups_places.head(10)

,GEOID_left,NAMELSAD_left,geometry,index_right,GEOID_right,NAME,NAMELSAD_right
1,171759514004,Block Group 4,"POLYGON ((-89.87840199999999 41.106688, -89.87...",992,1775783,Toulon,Toulon city
2,171759514003,Block Group 3,"POLYGON ((-89.98491399999999 41.084687, -89.98...",992,1775783,Toulon,Toulon city
5880,171759514005,Block Group 5,"POLYGON ((-89.88803999999999 41.098605, -89.88...",992,1775783,Toulon,Toulon city
2,171759514003,Block Group 3,"POLYGON ((-89.98491399999999 41.084687, -89.98...",975,1740676,La Fayette,La Fayette village
3,171759514001,Block Group 1,"POLYGON ((-89.810795 41.22781699999999, -89.81...",982,1707692,Bradford,Bradford village
5879,171759514002,Block Group 2,"POLYGON ((-89.801857 41.153655, -89.8018319999...",982,1707692,Bradford,Bradford village
5,170579538001,Block Group 1,"POLYGON ((-90.190433 40.39615, -90.19041399999...",375,1743055,Lewistown,Lewistown city
21,170579532004,Block Group 4,"POLYGON ((-90.22672899999999 40.400984, -90.22...",375,1743055,Lewistown,Lewistown city
22,170579538002,Block Group 2,"POLYGON ((-90.19066099999999 40.382395, -90.19...",375,1743055,Lewistown,Lewistown city
32,170579537002,Block Group 2,"POLYGON ((-90.221006 40.357376, -90.220332 40....",375,1743055,Lewistown,Lewistown city


In [31]:
df = pd.merge(acs5, blockgroups_places, left_on='bg_GEOID', right_on='GEOID_left')
df.head()

,censusgeo,GEO_ID,county_name,bg_GEOID,GEOID_left,NAMELSAD_left,geometry,index_right,GEOID_right,NAME,NAMELSAD_right
0,"[['Block Group 3', ' Census Tract 101', ' Je...",1500000US170830101003,' Jersey County',170830101003,170830101003,Block Group 3,"POLYGON ((-90.32592199999999 39.11707, -90.324...",1025,1738414,Jerseyville,Jerseyville city
1,"[['Block Group 4', ' Census Tract 101', ' Je...",1500000US170830101004,' Jersey County',170830101004,170830101004,Block Group 4,"POLYGON ((-90.363429 39.17442399999999, -90.34...",1025,1738414,Jerseyville,Jerseyville city
2,"[['Block Group 1', ' Census Tract 105', ' Je...",1500000US170830105001,' Jersey County',170830105001,170830105001,Block Group 1,"POLYGON ((-90.591702 39.086772, -90.5903149999...",1024,1730653,Grafton,Grafton city
3,"[['Block Group 2', ' Census Tract 102', ' Je...",1500000US170830102002,' Jersey County',170830102002,170830102002,Block Group 2,"POLYGON ((-90.21624399999999 39.00007, -90.216...",1022,1708264,Brighton,Brighton village
4,"[['Block Group 2', ' Census Tract 102', ' Je...",1500000US170830102002,' Jersey County',170830102002,170830102002,Block Group 2,"POLYGON ((-90.21624399999999 39.00007, -90.216...",806,1730094,Godfrey,Godfrey village


In [43]:
#Load transit score csv
ts = pd.read_csv('transit_score.csv')

In [44]:
tsdf = pd.merge(df, ts, how='inner', left_on='NAME', right_on='city')
tsdf.shape

(5281, 26)

In [45]:
tsdf.head()

,censusgeo_x,GEO_ID_x,county_name,bg_GEOID,GEOID_left,NAMELSAD_left,geometry,index_right,GEOID_right,NAME,...,nearby_routes,bus,rail,other,censusgeo_y,GEO_ID_y,Place_Type,state,Lat,Lon
0,"[['Block Group 4', ' Census Tract 14', ' DeK...",1500000US170370014004,' DeKalb County',170370014004,170370014004,Block Group 4,"POLYGON ((-88.8223 41.848823, -88.822147 41.84...",212,1719161,DeKalb,...,7,7,0,0,"DeKalb city, Illinois: Summary level: 160, sta...",1600000US1719161,city,IL,41.928368,-88.751178
1,"[['Block Group 3', ' Census Tract 14', ' DeK...",1500000US170370014003,' DeKalb County',170370014003,170370014003,Block Group 3,"POLYGON ((-88.773944 41.905048, -88.773786 41....",212,1719161,DeKalb,...,7,7,0,0,"DeKalb city, Illinois: Summary level: 160, sta...",1600000US1719161,city,IL,41.928368,-88.751178
2,"[['Block Group 2', ' Census Tract 14', ' DeK...",1500000US170370014002,' DeKalb County',170370014002,170370014002,Block Group 2,"POLYGON ((-88.770521 41.916687, -88.770214 41....",212,1719161,DeKalb,...,7,7,0,0,"DeKalb city, Illinois: Summary level: 160, sta...",1600000US1719161,city,IL,41.928368,-88.751178
3,"[['Block Group 1', ' Census Tract 8', ' DeKa...",1500000US170370008001,' DeKalb County',170370008001,170370008001,Block Group 1,"POLYGON ((-88.74395299999999 41.939105, -88.74...",212,1719161,DeKalb,...,7,7,0,0,"DeKalb city, Illinois: Summary level: 160, sta...",1600000US1719161,city,IL,41.928368,-88.751178
4,"[['Block Group 4', ' Census Tract 8', ' DeKa...",1500000US170370008004,' DeKalb County',170370008004,170370008004,Block Group 4,"POLYGON ((-88.745245 41.929572, -88.745149 41....",212,1719161,DeKalb,...,7,7,0,0,"DeKalb city, Illinois: Summary level: 160, sta...",1600000US1719161,city,IL,41.928368,-88.751178


In [55]:
tsdf.columns

Index(['censusgeo_x', 'GEO_ID_x', 'county_name', 'bg_GEOID', 'GEOID_left',
       'NAMELSAD_left', 'geometry', 'index_right', 'GEOID_right', 'NAME',
       'NAMELSAD_right', 'city', 'description', 'summary', 'transit_score',
       'ws_link', 'nearby_routes', 'bus', 'rail', 'other', 'censusgeo_y',
       'GEO_ID_y', 'Place_Type', 'state', 'Lat', 'Lon'],
      dtype='object')

In [57]:
tsdf[tsdf['city']=='Chicago'].shape

(2345, 26)

In [1]:
import geopandas as gpd
tracts = gpd.read_file('shape_tracts/tl_2018_17_tract.shp')

In [3]:
tracts = tracts[['GEOID', 'NAMELSAD', 'geometry']]

In [7]:
from download import acs5

Unexpected response (URL: https://api.census.gov/data/2018/acs/acs5?get=NAME&for=county:*&in=state:17): Sorry, the system is currently undergoing maintenance or is busy.  Please try again later. 


ValueError: 